In [ ]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
import datetime

In [ ]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
# conn = sqlite3.connect('mom.db')
# cur = conn.cursor()
# cur.executescript('''
#     DROP TABLE IF EXISTS head;
#     CREATE TABLE head(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         head TEXT NOT NULL,
#         cdate TEXT NOT NULL,
#         wdate TEXT NOT NULL,
#         ref INTEGER NOT NULL,
#         page INTEGER NOT NULL
#     );
# ''')

# cur.executescript('''
#     DROP TABLE IF EXISTS history;
#     CREATE TABLE history(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         seen TEXT NOT NULL,
#         ref INTEGER NOT NULL
#     );
# ''')

In [ ]:
# 처음 시작할 때 
# 맘스홀릭베이비 ref = 104
url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10094499&search.menuid=179&search.boardtype=L&search.totalCount=151&search.page=1'
seen = list()

urls = list()
urls.append(url)

count = 0

while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 104)
        ''', [seed])

        for _ in [_['href'] for _ in dom.select('div.prev-next > a')
                  if _.has_attr('href')and re.match(r'/ArticleList',_['href'])]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                 urls.append(newUrls)

        if dom.select('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article') != None:
            head = ["".join(re.findall(r'[^\n]+',_.text.strip())) for _ in dom.select('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')]
            wdate = [_.text.strip() for _ in dom.select('#main-area > div:nth-child(6) > table > tbody > tr > td.td_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,104)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)

In [ ]:
# 맘카페
# 이어서 시작할 때
conn = sqlite3.connect('mom.db')
cur = conn.cursor()

cur.execute('SELECT seen FROM history')
seen =cur.fetchall()
seen_ = list()
for _ in seen:
    seen_.append(''.join(_))
url = seen[-1]
url = ''.join(url)

In [ ]:
# 맘카페
# 이어서 시작할 때
urls = list()
urls.append(url)

count = 0

while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen_.append(seed)
        cur.execute('''
                    INSERT INTO history(seen, ref) VALUES(?, 104)
                ''', [seed])

        for _ in [_['href'] for _ in dom.select('div.prev-next > a')
                  if _.has_attr('href')and re.match(r'/ArticleList',_['href'])]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen_:
                urls.append(newUrls)
                            

        if dom.select('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article') != None:
            head = ["".join(re.findall(r'[^\n]+',_.text.strip())) for _ in dom.select('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')]
            wdate = [_.text.strip() for _ in dom.select('#main-area > div:nth-child(6) > table > tbody > tr > td.td_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,104)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)